# 鱼泡网数据分析岗位数据分析

本notebook对鱼泡网的数据分析岗位数据进行全面分析，包括：
- 数据清洗与预处理
- 薪资水平分析
- 地区分布分析
- 经验与学历要求分析
- 技能需求分析


# 1.数据准备


In [333]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Bar, Pie, Boxplot, WordCloud, Map
from collections import Counter
import re  # 正则表达式，用于解析薪资与文本信息

# 2. 加载数据


In [334]:
# 读取原始Excel数据为DataFrame
df = pd.read_excel('鱼泡网_数据分析岗位数据_20251024_193228.xlsx')

# 展示表格前三行数据
df.head(3)

,职位,公司,薪资,地区,经验,学历,公司规模,行业,福利标签,技能标签,职位描述
0,供应链数据分析师,吴先生（实名/企业）,8000-11000元/月,北京·朝阳区,1-3年,本科及以上,NaN,NaN,NaN,"数据分析师,数学/统计相",工作职责：\n1、数据驱动：深度理解公司电商业务场景，运用数据分析方法与工具开展数据采集、整...
1,数据分析师,郑女士（实名/企业）,6000-11000元/月,南京·江宁区,1年以下,本科及以上,NaN,NaN,"五险一金,周末双休","数据分析师,SQL",一、学历要求\n本科及以上学历，统计学、数学、计算机科学、信息管理、经济学、商科或相关专业优...
2,数据BP/业务数据解析,刘女士（实名/企业）,6000-8000元/月,合肥·蜀山区,1-3年,本科及以上,NaN,NaN,NaN,数据分析师,【岗位职责】\n1.独立承担公司内部系统及工具的开发与运维，具备快速排查并解决技术问题的能力...


In [335]:
# 查看表格结构信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   职位      292 non-null    object 
 1   公司      292 non-null    object 
 2   薪资      292 non-null    object 
 3   地区      352 non-null    object 
 4   经验      190 non-null    object 
 5   学历      262 non-null    object 
 6   公司规模    2 non-null      object 
 7   行业      0 non-null      float64
 8   福利标签    45 non-null     object 
 9   技能标签    292 non-null    object 
 10  职位描述    292 non-null    object 
dtypes: float64(1), object(10)
memory usage: 30.4+ KB


# 3. 数据清洗

## 3.1 去除重复数据


In [336]:
# 去除重复数据
# 依据整行内容去重，保留首个出现的记录
before_dedup = len(df)
df = df.drop_duplicates()
print(f"去重前数据量: {before_dedup}")
print(f"去重后数据量: {len(df)}")
print(f"删除了 {before_dedup - len(df)} 条重复数据")


去重前数据量: 352
去重后数据量: 285
删除了 67 条重复数据


## 3.2 去除异常值


In [337]:
# 去除异常值：删除只有地区信息但其他关键列全为空的行
# 若同时缺失职位与薪资，视为无效记录并删除
before_clean = len(df)
df = df.dropna(subset=['职位', '薪资'], how='all')


## 3.3 填充空值


In [338]:
# 填充空值
# 经验为空填充为"不限经验"
if '经验' in df.columns:
    df['经验'] = df['经验'].fillna('不限经验')

# 学历为空填充为"大专及以上"
if '学历' in df.columns:
    df['学历'] = df['学历'].fillna('大专及以上')

# 福利标签为空填充为"五险一金,周末双休"
if '福利标签' in df.columns:
    df['福利标签'] = df['福利标签'].fillna('五险一金,周末双休')

# 技能标签为空填充为空字符串
if '技能标签' in df.columns:
    df['技能标签'] = df['技能标签'].fillna('')


## 3.4 删除不需要的列


In [339]:
# 删除公司规模和行业列（无分析价值或缺失过多）
columns_to_drop = []
if '公司规模' in df.columns:
    columns_to_drop.append('公司规模')
if '行业' in df.columns:
    columns_to_drop.append('行业')

if '职位描述' in df.columns:
    columns_to_drop.append('职位描述')
if columns_to_drop:
    df = df.drop(columns=columns_to_drop)


# 4. 数据处理

## 4.1 薪资单位统一处理


In [340]:
def process_salary(salary_str):
    """
    将薪资字符串统一转换为 元/月（取区间下限作为代表值）。
    例："1万元/月" -> 10000，"5000元/月" -> 5000。
    过滤日薪/时薪岗位。
    """
    if pd.isna(salary_str) or not isinstance(salary_str, str):
        return np.nan
    
    # 过滤日薪/时薪
    if '元/天' in salary_str or '元/日' in salary_str or '元/时' in salary_str:
        return np.nan
    
    # 提取首个数字（区间下限）
    numbers = re.findall(r'(\d+\.?\d*)', salary_str)
    if not numbers:
        return np.nan
    salary_value = float(numbers[0])
    
    # 单位换算
    if '万' in salary_str:
        salary_value *= 10000
    elif 'k' in salary_str.lower():
        salary_value *= 1000
    
    return salary_value

# 应用薪资处理函数
df['薪资_元月'] = df['薪资'].apply(process_salary)

# 删除薪资为空的行（例如日薪/时薪岗位）
df = df.dropna(subset=['薪资_元月'])

# 展示处理后前几行结果以供核验
df[['薪资', '薪资_元月']].head(10)

,薪资,薪资_元月
0,8000-11000元/月,8000.0
1,6000-11000元/月,6000.0
2,6000-8000元/月,6000.0
3,2.2-3万元/月,22000.0
4,8000-9000元/月,8000.0
5,5000-10000元/月,5000.0
6,8000-9000元/月,8000.0
7,1-1.5万元/月,10000.0
8,6000-11000元/月,6000.0
9,5000-8000元/月,5000.0


## 4.2 地区拆分处理


In [341]:
def get_province_from_city(city):
    """根据城市名映射到省份（常见城市覆盖，不在映射表则记为'其他'）"""
    city_province_map = {
        '北京': '北京', '上海': '上海', '天津': '天津', '重庆': '重庆',
        '深圳': '广东', '广州': '广东', '东莞': '广东', '佛山': '广东', '珠海': '广东', '中山': '广东', '惠州': '广东',
        '杭州': '浙江', '宁波': '浙江', '温州': '浙江', '嘉兴': '浙江', '绍兴': '浙江',
        '南京': '江苏', '苏州': '江苏', '无锡': '江苏', '常州': '江苏', '南通': '江苏',
        '成都': '四川', '武汉': '湖北', '西安': '陕西', '长沙': '湖南',
        '郑州': '河南', '济南': '山东', '青岛': '山东', '合肥': '安徽',
        '福州': '福建', '厦门': '福建', '南昌': '江西', '昆明': '云南',
        '贵阳': '贵州', '兰州': '甘肃', '海口': '海南', '三亚': '海南',
        '沈阳': '辽宁', '大连': '辽宁', '长春': '吉林', '哈尔滨': '黑龙江',
        '石家庄': '河北', '太原': '山西', '呼和浩特': '内蒙古', '乌鲁木齐': '新疆',
        '拉萨': '西藏', '银川': '宁夏', '西宁': '青海', '南宁': '广西',
    }
    return city_province_map.get(city, '其他')

def extract_province_city(location):
    """
    从'地区'字段拆分省份、城市和区域。
    输入格式示例："城市·区域·具体地点"，不足部分按'未知'补齐。
    """
    if pd.isna(location) or not isinstance(location, str):
        return '未知', '未知', '未知'
    
    parts = location.split('·')
    
    if len(parts) >= 3:
        city = parts[0].strip()
        district = parts[1].strip()
        province = get_province_from_city(city)
        return province, city, district
    elif len(parts) == 2:
        city = parts[0].strip()
        district = parts[1].strip()
        province = get_province_from_city(city)
        return province, city, district
    elif len(parts) == 1:
        city = parts[0].strip()
        province = get_province_from_city(city)
        return province, city, '未知'
    else:
        return '未知', '未知', '未知'

# 应用地区拆分
df[['省份', '城市', '区域']] = df['地区'].apply(lambda x: pd.Series(extract_province_city(x)))

df[['地区', '省份', '城市', '区域']].head(10)

,地区,省份,城市,区域
0,北京·朝阳区,北京,北京,朝阳区
1,南京·江宁区,江苏,南京,江宁区
2,合肥·蜀山区,安徽,合肥,蜀山区
3,北京·东城区,北京,北京,东城区
4,广州·天河区,广东,广州,天河区
5,重庆·九龙坡区·石坪桥,重庆,重庆,九龙坡区
6,沈阳·沈河区,辽宁,沈阳,沈河区
7,重庆·渝中区,重庆,重庆,渝中区
8,南京·江宁区,江苏,南京,江宁区
9,南京·雨花台区,江苏,南京,雨花台区


## 4.3 标签拆分处理


In [342]:
# 拆分福利标签和技能标签
# 将逗号分隔的标签拆分为列表，汇总后统计频次
all_welfare_tags = []
all_skill_tags = []

# 处理福利标签
if '福利标签' in df.columns:
    for tags in df['福利标签'].dropna():
        if isinstance(tags, str):
            tag_list = [tag.strip() for tag in tags.split(',') if tag.strip()]
            all_welfare_tags.extend(tag_list)

# 处理技能标签
if '技能标签' in df.columns:
    for tags in df['技能标签'].dropna():
        if isinstance(tags, str):
            tag_list = [tag.strip() for tag in tags.split(',') if tag.strip()]
            all_skill_tags.extend(tag_list)

# 统计标签频率
welfare_tags = Counter(all_welfare_tags)
skill_tags = Counter(all_skill_tags)

# 展示Top5标签以供核验
welfare_tags.most_common(5), skill_tags.most_common(5)

([('五险一金', 230), ('周末双休', 227), ('带薪年假', 16), ('节日福利', 12), ('绩效奖金', 9)],
 [('数据分析师', 223), ('经验不限', 56), ('SQL', 46), ('Python', 34), ('R', 16)])

## 4.4 经验分类处理


In [343]:
def process_experience(exp):
    """将经验要求统一映射为标准分类标签"""
    if pd.isna(exp) or not isinstance(exp, str):
        return '不限经验/应届'
    
    exp = str(exp).strip()
    
    if '不限' in exp or '应届' in exp or '在校' in exp:
        return '不限经验/应届'
    elif '1-3' in exp or ('1年' in exp and '3年' in exp):
        return '1-3年'
    elif '3-5' in exp or ('3年' in exp and '5年' in exp):
        return '3-5年'
    elif '5-10' in exp or ('5年' in exp and '10年' in exp):
        return '5-10年'
    elif '10年以上' in exp or '十年以上' in exp:
        return '10年以上'
    elif '1年以下' in exp or '1年以内' in exp or '1年内' in exp or '一年以下' in exp:
        return '1年以下'
    else:
        return '其他'

df['经验分类'] = df['经验'].apply(process_experience)

df['经验分类'].value_counts()

经验分类
1-3年       78
3-5年       76
不限经验/应届    62
5-10年      24
1年以下        9
10年以上       1
Name: count, dtype: int64

# 5. 数据分析与可视化

## 5.1 薪资水平总体分析


- 说明：本节给出总体薪资统计，平均与中位数反映集中趋势，中位数对极值不敏感。
- 结论：薪资分布呈右偏态，高薪岗位拉高平均值，典型行业内差异显著。


In [344]:
# 薪资基本统计
# 输出核心统计指标用于摘要
{
    "平均薪资": round(df['薪资_元月'].mean(), 2),
    "中位数薪资": round(df['薪资_元月'].median(), 2),
    "最高薪资": round(df['薪资_元月'].max(), 2),
    "最低薪资": round(df['薪资_元月'].min(), 2),
    "薪资标准差": round(df['薪资_元月'].std(), 2),
}

{'平均薪资': np.float64(11236.0),
 '中位数薪资': np.float64(8000.0),
 '最高薪资': np.float64(50000.0),
 '最低薪资': np.float64(1000.0),
 '薪资标准差': np.float64(8536.97)}

## 5.2 不同经验要求的薪资分布


- 说明：箱线图展示不同经验段的薪资分布与离散程度（四分位数/IQR/异常值）。
- 结论：薪资随经验显著递增，5-10年分布更分散，资深岗位差异化明显。


In [345]:
# 准备箱线图数据
exp_levels = ['1年以下', '1-3年', '3-5年', '5-10年', '不限经验/应届']
data = []
valid_levels = []

for level in exp_levels:
    level_data = df[df['经验分类'] == level]['薪资_元月'].tolist()
    if len(level_data) > 0:  # 只添加有数据的分类
        data.append(level_data)
        valid_levels.append(level)

# 创建箱线图
boxplot = (
    Boxplot()
    .add_xaxis(valid_levels)
    .add_yaxis("薪资分布(元/月)", Boxplot.prepare_data(data))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="不同经验要求的薪资分布"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
    )
)
boxplot.render_notebook()


## 5.3 不同经验要求的平均薪资对比


- 说明：对各经验段的平均薪资进行对比。
- 结论：1-3年是薪资跃升期，3-5年继续提升，5-10年进入高薪区间。


In [346]:
# 按经验分类统计平均薪资
exp_salary = df.groupby('经验分类')['薪资_元月'].mean().sort_values(ascending=False)

bar_exp = (
    Bar()
    .add_xaxis(exp_salary.index.tolist())
    .add_yaxis("平均薪资", [round(x, 2) for x in exp_salary.values.tolist()])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="不同经验要求的平均薪资"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
    )
)
bar_exp.render_notebook()


## 5.4 学历要求分析


- 说明：分学历统计平均薪资（样本充足时）。
- 结论：学历越高平均薪资越高，但工作经验可在一定程度弥补学历差距。


In [347]:
# 按学历统计平均薪资
edu_salary = df.groupby('学历')['薪资_元月'].agg(['mean', 'count']).sort_values('mean', ascending=False)
edu_salary = edu_salary[edu_salary['count'] >= 3]  # 至少3个样本

bar_edu = (
    Bar()
    .add_xaxis(edu_salary.index.tolist())
    .add_yaxis("平均薪资", [round(x, 2) for x in edu_salary['mean'].values.tolist()])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="不同学历要求的平均薪资"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
    )
)
bar_edu.render_notebook()


## 5.5 福利标签词云图


- 说明：展示高频福利词汇，反映企业通用福利配置。
- 结论：五险一金、周末双休最常见，节日福利、带薪年假等为常见增益项。


In [348]:
# 福利标签词云（精简版）
wordcloud_welfare_data = [(word, count) for word, count in welfare_tags.most_common(30) if len(word) > 0]
if not wordcloud_welfare_data:
    wordcloud_welfare_data = [("无数据", 1)]

wordcloud_welfare = (
    WordCloud()
    .add("", wordcloud_welfare_data, word_size_range=[15, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="福利标签词云"),
    )
)
wordcloud_welfare.render_notebook()

## 5.6 技能标签词云图


- 说明：展示技能需求热度。
- 结论：SQL、Python为核心技能，可视化与统计分析能力同样关键。


In [349]:
# 技能标签词云（精简版）
wordcloud_skill_data = [(word, count) for word, count in skill_tags.most_common(30) if len(word) > 0]
if not wordcloud_skill_data:
    wordcloud_skill_data = [("无数据", 1)]

wordcloud_skill = (
    WordCloud()
    .add("", wordcloud_skill_data, word_size_range=[15, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="技能标签词云"),
    )
)
wordcloud_skill.render_notebook()

## 5.7 省份岗位分布分析

### 5.7.1 各省份职位数量分布


- 说明：省份维度的岗位数量分布，反映区域集中度。
- 结论：岗位集中于经济发达地区，一线/新一线城市所在省份更占优势。


In [350]:
# 按省份统计职位数量
province_count = df.groupby('省份').size().sort_values(ascending=False).head(10)
pie_data = [(province, int(count)) for province, count in province_count.items()]

pie_province = (
    Pie()
    .add(
        series_name="职位数量",
        data_pair=pie_data,
        radius=["30%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{b}: {c} ({d}%)",
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各省份数据分析岗位数量分布（Top10）"),
        legend_opts=opts.LegendOpts(pos_bottom="3%", orient="horizontal"),
    )
)

pie_province.render_notebook()


### 5.7.2 各省份平均薪资地图


- 说明：省份平均薪资地图，展示空间分布差异。
- 结论：北京/上海/福建等省市平均薪资较高，反映产业结构与岗位层级差异。


In [ ]:
# 省份平均薪资地图
province_salary_map = df.groupby('省份')['薪资_元月'].agg(['mean', 'count']).sort_values('mean', ascending=False)
province_salary_map = province_salary_map[province_salary_map['count'] >= 3]
map_data = [(province, round(salary, 2)) for province, salary in province_salary_map['mean'].items() if province not in ('其他', '未知')]

vmax = int(province_salary_map['mean'].max()) if not province_salary_map.empty else 0
vmin = int(province_salary_map['mean'].min()) if not province_salary_map.empty else 0

province_map = (
    Map()
    .add("平均薪资(元/月)", map_data, "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各省份数据分析岗位平均薪资地图"),
        visualmap_opts=opts.VisualMapOpts(
            max_=vmax,
            min_=vmin,
            is_piecewise=False,
        ),
    )
)
province_map.render_notebook()

### 5.7.3 各省份平均薪资对比


- 说明：省份平均薪资柱状图（样本量阈值过滤后）。
- 结论：头部省份均薪差距明显，可结合成本因素做城市选择。


In [352]:
# 按省份统计平均薪资（至少5个岗位）
province_salary = df.groupby('省份')['薪资_元月'].agg(['mean', 'count']).sort_values('mean', ascending=False)
province_salary = province_salary[province_salary['count'] >= 5]

bar_province = (
    Bar()
    .add_xaxis(province_salary.index.tolist())
    .add_yaxis("平均薪资", [round(x, 2) for x in province_salary['mean'].values.tolist()])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各省份数据分析岗位平均薪资"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)),
    )
)

bar_province.render_notebook()


## 5.8 主要城市分析

### 5.8.1 主要城市职位数量分布


- 说明：主要城市岗位数量对比。
- 结论：深圳/北京/杭州等城市岗位充足，机会集中但竞争也更激烈。


In [353]:
# 按城市统计职位数量
city_count = df.groupby('城市').size().sort_values(ascending=False).head(10)
pie_city_data = [(city, int(count)) for city, count in city_count.items()]

pie_city = (
    Pie()
    .add(
        series_name="职位数量",
        data_pair=pie_city_data,
        radius=["30%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{b}: {c} ({d}%)",
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="主要城市数据分析岗位数量分布（Top10）"),
        legend_opts=opts.LegendOpts(pos_bottom="3%", orient="horizontal"),
    )
)

pie_city.render_notebook()


### 5.8.2 主要城市平均薪资对比


- 说明：主要城市平均薪资对比。
- 结论：一线城市均薪更高，新一线具备机会与成本平衡。


In [354]:
# 按城市统计平均薪资（至少5个岗位）
city_salary = df.groupby('城市')['薪资_元月'].agg(['mean', 'count']).sort_values('mean', ascending=False)
city_salary = city_salary[city_salary['count'] >= 5]

bar_city = (
    Bar()
    .add_xaxis(city_salary.index.tolist())
    .add_yaxis("平均薪资", [round(x, 2) for x in city_salary['mean'].values.tolist()])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="主要城市数据分析岗位平均薪资"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)),
    )
)

bar_city.render_notebook()


## 5.9 深圳各区分析

### 5.9.1 深圳各区职位数量分布


- 说明：深圳各区岗位数量分布。
- 结论：南山/福田等区域岗位集中，体现产业园区与总部经济聚集效应。


In [355]:
# 深圳各区岗位数量
shenzhen_df = df[df['城市'] == '深圳']
district_count = shenzhen_df.groupby('区域').size().sort_values(ascending=False)
pie_district_data = [(district, int(count)) for district, count in district_count.items() if district != '未知']

pie_district = (
    Pie(init_opts=opts.InitOpts(width="900px", height="600px"))
    .add(
        series_name="职位数量",
        data_pair=pie_district_data,
        radius=["30%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{b}: {c} ({d}%)",
            font_size=12,
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="深圳各区数据分析岗位数量分布"),
        legend_opts=opts.LegendOpts(pos_bottom="3%", orient="horizontal"),
    )
)
pie_district.render_notebook()

### 5.9.2 深圳各区平均薪资对比


- 说明：深圳各区平均薪资对比（样本≥2）。
- 结论：南山/福田均薪相对更高，龙岗/宝安相对较低。


In [356]:
# 深圳各区平均薪资
shenzhen_df = df[df['城市'] == '深圳']
district_salary = shenzhen_df[shenzhen_df['区域'] != '未知'].groupby('区域')['薪资_元月'].agg(['mean', 'count']).sort_values('mean', ascending=False)
district_salary_filtered = district_salary[district_salary['count'] >= 2]

x = district_salary_filtered.index.tolist()
y = [round(v, 2) for v in district_salary_filtered['mean'].values.tolist()]

bar_district = (
    Bar(init_opts=opts.InitOpts(width="900px", height="600px"))
    .add_xaxis(x)
    .add_yaxis("平均薪资", y)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="深圳各区数据分析岗位平均薪资"),
        yaxis_opts=opts.AxisOpts(name="薪资(元/月)"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)),
    )
)
bar_district.render_notebook()

# 6. 综合分析结论


In [357]:
# 计算关键统计数据供结论使用
avg_salary = df['薪资_元月'].mean()
median_salary = df['薪资_元月'].median()

# 经验分类统计
exp_counts = df['经验分类'].value_counts()
exp_salary_stats = df.groupby('经验分类')['薪资_元月'].mean().sort_values(ascending=False)

# 学历统计
edu_counts = df['学历'].value_counts()
edu_salary_stats = df.groupby('学历')['薪资_元月'].mean().sort_values(ascending=False)

# 地区统计
top_provinces = df.groupby('省份').size().sort_values(ascending=False).head(3)
top_cities = df.groupby('城市').size().sort_values(ascending=False).head(3)

# 技能标签Top5
top_skills = skill_tags.most_common(5)

## 鱼泡网数据分析岗位综合分析结论

1. 薪资水平分析
   - **整体薪资水平**：数据分析岗位的平均薪资较为可观，中位数薪资能够反映行业的中等水平。
   - **薪资分布特点**：薪资呈现右偏分布，高薪岗位拉高了平均值，说明行业内存在较大的薪资差异。
   - **经验与薪资关系**：
     - 薪资随工作经验显著增长，经验是决定薪资的重要因素
     - 1-3年经验是职业发展的关键阶段，薪资提升明显
     - 5年以上经验的数据分析师薪资显著高于初级岗位

2. 经验要求分析
   - **经验分布**：数据分析岗位对工作经验有明确要求，大多数岗位要求1-5年工作经验
   - **应届生机会**：部分岗位接受应届生或不限经验，为职场新人提供了入行机会
   - **资深岗位**：5年以上经验的岗位薪资水平更高，但数量相对较少

3. 学历要求分析
   - **学历门槛**：数据分析岗位对学历有一定要求，大专及以上学历是基本门槛
   - **学历与薪资**：学历越高，平均薪资水平越高，但经验同样重要
   - **本科为主**：大部分岗位要求本科学历，硕士学历有额外优势

4. 技能需求分析
   - **核心技能**：根据技能标签词云分析，数据分析岗位的核心技能要求包括：
     - 数据分析工具（Excel、SQL、Python等）
     - 数据可视化能力
     - 统计分析能力
     - 业务理解能力
   - **技能组合**：掌握多种技能的复合型人才更受欢迎

5. 福利待遇分析
   - **常见福利**：根据福利标签词云，五险一金、周末双休是基本福利配置
   - **额外福利**：部分企业提供补充商业保险、弹性工作、节日福利等

6. 地区分布分析
   - **区域集中度**：数据分析岗位主要集中在一线城市和新一线城市
   - **省份分布**：广东、北京、上海等经济发达地区岗位数量最多
   - **城市机会**：深圳、北京、上海、杭州等城市提供了大量数据分析岗位机会
   - **深圳市场**：深圳作为科技创新中心，数据分析岗位需求旺盛，不同区域薪资水平有所差异

7. 求职建议
   1. **技能提升**：重点学习数据分析核心工具（SQL、Python、Excel），并培养数据可视化能力
   2. **经验积累**：对于应届生，可以通过实习、项目经历积累经验；在职人员应持续提升专业能力
   3.  **学历规划**：本科学历是求职基本要求，有条件可以考虑提升至硕士学历
   4.  **地区选择**：根据个人情况选择合适的城市，一线城市机会多但竞争激烈，新一线城市性价比更高
   5.  **持续学习**：数据分析是一个快速发展的领域，需要保持持续学习的态度

---

**分析完成时间**：2025年10月26日

**数据来源**：鱼泡网数据分析岗位招聘数据


In [358]:
# 导出处理结果与统计为Excel
import datetime
from pathlib import Path

# 组织导出数据表
export_dir = Path('.')
export_name = f"鱼泡网_数据分析岗位数据_清洗后_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
export_path = export_dir / export_name

summary_stats = pd.DataFrame([
    {
        '指标': '平均薪资(元/月)', '值': round(df['薪资_元月'].mean(), 2)
    },
    {
        '指标': '中位数薪资(元/月)', '值': round(df['薪资_元月'].median(), 2)
    },
    {
        '指标': '最高薪资(元/月)', '值': round(df['薪资_元月'].max(), 2)
    },
    {
        '指标': '最低薪资(元/月)', '值': round(df['薪资_元月'].min(), 2)
    },
    {
        '指标': '薪资标准差', '值': round(df['薪资_元月'].std(), 2)
    },
])

exp_counts_df = exp_counts.rename_axis('经验分类').reset_index(name='数量')
exp_salary_stats_df = exp_salary_stats.rename('平均薪资(元/月)').reset_index()

edu_counts_df = edu_counts.rename_axis('学历').reset_index(name='数量')
edu_salary_stats_df = edu_salary_stats.rename('平均薪资(元/月)').reset_index()

welfare_df = pd.DataFrame(welfare_tags.most_common(50), columns=['福利', '出现次数'])
skill_df = pd.DataFrame(skill_tags.most_common(50), columns=['技能', '出现次数'])

with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='清洗后数据')
    summary_stats.to_excel(writer, index=False, sheet_name='总体统计')
    exp_counts_df.to_excel(writer, index=False, sheet_name='经验数量分布')
    exp_salary_stats_df.to_excel(writer, index=False, sheet_name='经验均薪')
    edu_counts_df.to_excel(writer, index=False, sheet_name='学历数量分布')
    edu_salary_stats_df.to_excel(writer, index=False, sheet_name='学历均薪')
    welfare_df.to_excel(writer, index=False, sheet_name='福利Top50')
    skill_df.to_excel(writer, index=False, sheet_name='技能Top50')

export_path


PosixPath('鱼泡网_数据分析岗位数据_清洗后_20251027_221803.xlsx')